In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import itertools
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib import cm
from matplotlib.lines import Line2D
from matplotlib.patches import Rectangle
from matplotlib.patches import FancyArrow
from progressbar import ProgressBar
sns.set(style='ticks', font='Arial')

# Methods section 2.7
## parse sample info files & define global variables
### _s: DS1
### _h: DS2

In [ ]:
# parse table of sample descriptions
strains_h = pd.read_csv("/home/mathieu/mhenault_landrylab/md_ty_expression/script/fig_md/Strain_info_hovhan.csv", index_col=0, sep=";")
strains_h['rep'] = strains_h['ID'].apply(lambda x: x.split('_')[-1])
strains_s = pd.read_csv("/home/mathieu/mhenault_landrylab/md_ty_expression/script/fig_md/strain_info.csv", index_col=0, sep=";")

In [ ]:
# define lists of TEs for each dataset
TE_H = ['TY1-I', 'TY2-I', 'TY3-I', 'TY4-I', 'Tsu4-bayanus-I']
TE_S = ['TY1-I-LTR-cer', 'TY2-I-LTR-cer', 'TY3-I-LTR-cer', 'TY4-I-LTR-cer', 'TY3-I-LTR-par','TY1-I-LTR-par', 'TY5-I-LTR-par', 'Tsu4-I-LTR-bay']

# input size of TE references; format is (start_internal, end_internal, start_ltr, end_ltr)
TE_size_h = dict(zip(TE_H, 
                [[1,5249,5249,5249], [1,5295,5295,5295], [1,4671,4671,4671], [1,5484,5484,5484],  [1,5453,5453,5453]]))
TE_size_s = dict(zip(TE_S, 
                [[1,5249,5250,5587],[1,5295,5296,5627],[1,4671,4672,5011],[1,5484,5485,5855],[1,4674,4675,5017],[1,5250,5251,5568],[1,4877,4878,5129],[1,5453,5454,5774]]))


In [ ]:
# input coordinates for annotations of each ty family
Coords = pd.read_csv('/home/mathieu/mhenault_landrylab/md_ty_expression/db/ltr_cds_coords.csv', index_col=0)
# Coords initially produced for DS3 analysis with 5' LTR. 
# since 5' LTR missing, substract from all relevant values
for f in ['cds_end','gag_end','cds_start']:
    Coords[f] = Coords[f]-Coords['ltr_end']
Coords['total_end'] = Coords['ltr_end'] + Coords['internal_len']
Coords.index = Coords['alias'].values
Coords = Coords.drop('alias', axis=1)

In [ ]:
# read coverage for Tys; bed files produced by samtools depth

COV_WS_H={}
idx = 0
with ProgressBar(max_value=strains_h.shape[0]) as bar:
    for s in strains_h.index:       
        
        depth = pd.read_csv(f"/home/mathieu/mhenault_landrylab/md_ty_expression/bed_md/{s}_Aligned.sortedByCoord.out.ty.bed",
                            sep='\t', header=None, dtype={0:str, 1:np.int16, 2:np.int16})
        depth['strain'] = np.repeat(s, depth.shape[0])
        depth = depth.astype({0:str,1:int,2:int,'strain':str})
        COV_WS_H[s] = {k: v for k,v in depth.groupby(0) if k in TE_H}
        
        idx += 1
        bar.update(idx)

COV_WS_S={}
idx = 0
with ProgressBar(max_value=strains_s.shape[0]) as bar:
    for s in strains_s.index:       

        depth = pd.read_csv(f"/home/mathieu/mhenault_landrylab/md_ty_expression/bed_md/{s}_TY.tot.sorted.ty.bed",
                            sep='\t', header=None, dtype={0:str, 1:np.int16, 2:np.int16})
        depth['strain'] = np.repeat(s, depth.shape[0])
        depth = depth.astype({0:str,1:int,2:int,'strain':str})
        COV_WS_S[s] = {k: v for k,v in depth.groupby(0) if k in TE_S}

        idx += 1
        bar.update(idx)

In [ ]:
#import TPM
TPM = []
for file, ds in zip(['raw_counts_tot_cer_par_TPM.txt', 'raw_counts_tot_cer_bay_TPM.txt', 'counts_hovhan_cer_bay_TPM.txt'],
                   ['S','S','H']):

    tpm = pd.read_csv(f'/home/mathieu/mhenault_landrylab/md_ty_expression/script/fig_md/{file}', sep='\t')
    tpm = pd.melt(tpm, id_vars=['gene','length'], var_name='species', value_name='tpm')
    tpm['ds'] = ds
    

    if ds == 'H':
        tpm['species'] = tpm['species'].apply(lambda x: f'map_{x.split("_")[1]}')
        tpm['gene'] = np.where(tpm['gene'].apply(lambda x: 'Tsu4' in x),
                               tpm['gene'].apply(lambda x: x.replace('b','bayanus')),
                               tpm['gene'].apply(lambda x: x.replace('_','-')))

    if ds == 'S':
        tpm['species'] = tpm['species'].apply(lambda x: x.split("_")[0])
        tpm['gene'] = tpm['gene'].apply(lambda x: x.replace('_', '-I-LTR-'))

    tpm.index = pd.MultiIndex.from_frame(tpm[['gene','species']])
    TPM.append(tpm)
TPM = pd.concat(TPM)
TPM['log_tpm'] = np.log2(TPM['tpm']+1)

In [ ]:
#plot z-scores for bins along reference sequences - Fig 2
fig = plt.figure(figsize=[7.8,6])
gs = plt.GridSpec(nrows=2, ncols=3, hspace=0.47, wspace=0.7, left=0.12, right=0.96, top=0.94, bottom=0.14)

fam_alias_h = dict(zip(TE_H, ["Ty1_cer","Ty2_cer","Ty3_cer","Ty4_cer","Tsu4_uva"]))
# bin width:
wdw = 75
for idx, t in enumerate(TE_H):

    S = [s for s in strains_h.index if t in COV_WS_H[s]]
    data = pd.concat([COV_WS_H[s][t] for s in S], axis=0)
    data['bin'] = pd.cut(data[1], bins=pd.interval_range(start=1, end=data[1].max(), freq=wdw, closed='left'))
    data = data.groupby(['strain','bin'])[2].apply(lambda x: x.fillna(0).mean()).reset_index()
    
    for s, df in data.groupby('strain'):
        std = np.nanstd(df[2])

        if std==0: # means there is no data; set z-score to zero to avoid inf
    
            data.loc[df.index, 'zscore'] = np.nan #np.where(np.invert(np.isnan(df[2])), 0, np.nan)

        else:
            data.loc[df.index, 'zscore'] = (df[2]-np.nanmean(df[2]))/std
    data = pd.pivot_table(data, index='strain', columns='bin', values='zscore')

    # fill missing data with 0
    S_zero = [s for s in strains_h.index if t not in COV_WS_H[s]]
    for s in S_zero :
        data.loc[s] = np.nan
    # print quantiles for robust colormap visualization
    #print(np.quantile(data.values.reshape(1,-1), 0.98))
    #print(np.quantile(data.values.reshape(1,-1), 0.02))
    
    # sort data according to the plotting order wanted
    sort_index = strains_h.sort_values(by=['temperature','species','rep'], ascending=[False, False, True]).index
    data = data.loc[sort_index]
    
    #initiate plot
    ax = fig.add_subplot(gs[idx])

    HM = ax.imshow(data, cmap='viridis', aspect='auto', interpolation='none', vmin=-1.8, vmax=4.7)
    
    # add TPM
    tpm = TPM.loc[TPM['ds']=='H']
    ax.text(1.02, 18.25/18, 'log2\nTPM', va='bottom', size=6, transform=ax.transAxes)
    for pos, s in enumerate(sort_index):
        lc = tpm.loc[(t,s), 'log_tpm']
        ax.text(1.02, (17.5-pos)/18, f'{lc:.2f}', va='center', size=6, transform=ax.transAxes)
    
    fig.canvas.draw()
    tl = [int(t.get_text().replace('−','-'))*wdw for t in ax.get_xticklabels()]
    ax.set_xticklabels(tl)
    
    # plot TE symbol below heatmap
    
    base_below = data.shape[0]-1
    unit_below = 0.7
    
    xrange = ax.axis()[:2]
    coords = Coords.loc[fam_alias_h[t]]
    slope = (xrange[1]-xrange[0])/coords['internal_len']
    intercept = xrange[1]-(coords['internal_len']*slope)
    get_data_coord = np.vectorize(lambda x: x*slope+intercept)
    coords = get_data_coord(coords)
    
    # add patch for internal sequence
    internal = Rectangle([-0.5, base_below+7*unit_below], coords[5], unit_below, color='0.6', zorder=0, clip_on=False)
    ax.add_patch(internal)
    pol = FancyArrow(coords[4], base_below+6*unit_below, coords[1]-coords[4], 0, clip_on=False, zorder=1, width=unit_below, head_width=unit_below, fc='w', ec='k', lw=0.5,
                     length_includes_head=True)
    ax.add_patch(pol)
    ax.text(0.5*(coords[1]-coords[4])-0.5, base_below+6*unit_below+0.05, '$POL$', va='center', ha='center', color='k', zorder=2, size=5)
    
    if 'Ty5' not in t:
    # add line for GAG
        pol = FancyArrow(coords[4], base_below+4.5*unit_below, coords[2]-coords[4], 0, clip_on=False, zorder=1, width=unit_below, head_width=unit_below, fc='w', ec='k', lw=0.5,
                        length_includes_head=True)
        ax.add_patch(pol)
        ax.text(0.5*(coords[2]-coords[4])-0.5, base_below+4.5*unit_below+0.05, '$GAG$', va='center', ha='center', color='k', zorder=2, size=5)
    
    #for sp in ['left','right', 'top', 'bottom']:
    #    ax.spines[sp].set_visible(False)
    
    ax.set_title(fam_alias_h[t], size=10)
    ax.set_xticks([])
    ax.set_yticks(range(18))
    ax.set_yticklabels(strains_h['name'], size=6)
    
    ax.set_ylim(17.5, -0.5)

    ax.set_xticks(np.arange(0,5000,1500)/wdw)
    ax.set_xticklabels(np.arange(0,5000,1500), size=8)
    
cbar_ax = fig.add_axes([0.75, 0.28, 0.2, 0.02])
cbar = plt.colorbar(HM, cax=cbar_ax, orientation='horizontal', label='coverage depth\nz-score')
cbar.set_ticks(np.arange(-1,4,2))
cbar.outline.set_visible(False)


plt.savefig('/home/mathieu/mhenault_landrylab/md_ty_expression/fig/fig_final/Fig2B.jpg', dpi=300)

plt.show()
plt.close()

In [ ]:
# Same code as previous cell, but for DS1
fig = plt.figure(figsize=[7.8,6])
gs = plt.GridSpec(nrows=3, ncols=3, height_ratios=[1,1,0.5], hspace=0.85, wspace=0.7, left=0.12, right=0.96, top=0.94, bottom=0.13)
replicate = ['map_51', 'map_52', 'map_53', 'map_54','map_55', 'map_56', 'map_57', 'map_58','map_59','map_60','map_61'
             'map_62', 'map_63', 'map_64', 'map_65', 'map_66', 'map_67', 'map_68']
fam_alias_s = dict(zip(TE_S, ['Ty1_cer', 'Ty2_cer','Ty3_cer','Ty4_cer', 'Ty3_par', 'Ty1_par', 'Ty5_par', 'Tsu4_uva']))
# bin width
wdw = 75
for idx, t in zip([0,1,2,3,4,6,7,8], TE_S):
    

    S = [s for s in strains_s.index if t in COV_WS_S[s]]
    data = pd.concat([COV_WS_S[s][t] for s in S], axis=0)

    data['bin'] = pd.cut(data[1], bins=pd.interval_range(start=1, end=data[1].max(), freq=wdw, closed='left'))
    data = data.groupby(['strain','bin'])[2].apply(lambda x: x.fillna(0).mean()).reset_index()

    for s, df in data.groupby('strain'):
        std = np.nanstd(df[2])

        if std==0:
            data.loc[df.index, 'zscore'] = np.nan

        else:
            data.loc[df.index, 'zscore'] = (df[2]-np.nanmean(df[2]))/std
    
    data = pd.pivot_table(data, index='strain', columns='bin', values='zscore')

    #print(np.quantile(data.values.reshape(1,-1), 0.98))
    #print(np.quantile(data.values.reshape(1,-1), 0.02))
    
    S_zero = [s for s in strains_s.index if s not in data.index]
    for s in S_zero :
        data.loc[s]= np.nan
    
    # sort data according to the plotting order wanted
    sort_index = strains_s.sort_values(by=['cross_order','type','genotype'], ascending=[True, False, True]).index
    if t in ['TY1-I-LTR-par', 'TY5-I-LTR-par']:
        sort_index = sort_index[:6]
    if t == 'Tsu4-I-LTR-bay':
        sort_index = sort_index[6:]
    data = data.loc[sort_index]

    
    #initiate plot
    ax = fig.add_subplot(gs[idx])

    HM = ax.imshow(data, cmap='viridis', aspect='auto', interpolation='none', vmin=-1.8, vmax=4.7)
    
    # add TPM
    tpm = TPM.loc[TPM['ds']=='S']
    ax.text(1.02, (data.shape[0]+0.25)/data.shape[0], 'log2\nTPM', va='bottom', size=6, transform=ax.transAxes)
    for pos, s in enumerate(sort_index):
        #if s in tpm.loc[t].index:
        lc = tpm.loc[(t,s), 'log_tpm']
        ax.text(1.02, (data.shape[0]-0.5-pos)/data.shape[0], f'{lc:.2f}', va='center', size=6, transform=ax.transAxes)

    fig.canvas.draw()
    tl = [int(t.get_text().replace('−','-'))*wdw for t in ax.get_xticklabels()]
    ax.set_xticklabels(tl)
    
    # plot TE symbol below heatmap
    
    base_below = data.shape[0]-1
    unit_below = 0.7
    
    xrange = ax.axis()[:2]
    coords = Coords.loc[fam_alias_s[t]]
    slope = (xrange[1]-xrange[0])/coords['total_end']
    intercept = xrange[1]-(coords['total_end']*slope)
    get_data_coord = np.vectorize(lambda x: x*slope+intercept)
    coords = get_data_coord(coords)
    
    # add patch for internal sequence
    internal = Rectangle([-0.5, base_below+7*unit_below], coords[5], unit_below, color='0.6', zorder=0, clip_on=False)
    ax.add_patch(internal)
    # add patch for three prime LTR
    ltr = Rectangle([coords[5]-0.5, base_below+7*unit_below], coords[0], unit_below, color='0.3', zorder=0, clip_on=False)
    ax.add_patch(ltr)
    # add line for POL
    pol = FancyArrow(coords[4]-0.5, base_below+6*unit_below, coords[1]-coords[4], 0, clip_on=False, zorder=1, width=unit_below, head_width=unit_below, fc='w', ec='k', lw=0.5,
                     length_includes_head=True)
    ax.add_patch(pol)
    ax.text(0.5*(coords[1]-coords[4])-0.5, base_below+6*unit_below+0.05, '$POL$', va='center', ha='center', color='k', zorder=2, size=5)
    
    if 'Ty5' not in t:
    # add line for GAG
        pol = FancyArrow(coords[4]-0.5, base_below+4.5*unit_below, coords[2]-coords[4], 0, clip_on=False, zorder=1, width=unit_below, head_width=unit_below, fc='w', ec='k', lw=0.5,
                        length_includes_head=True)
        ax.add_patch(pol)
        ax.text(0.5*(coords[2]-coords[4])-0.5, base_below+4.5*unit_below+0.05, '$GAG$', va='center', ha='center', color='k', zorder=2, size=5)
    
    #for sp in ['left','right', 'top', 'bottom']:
    #    ax.spines[sp].set_visible(False)
  
    ax.set_title(fam_alias_s[t], size=10)
    ax.set_xticks([])
    ax.set_yticks(range(data.shape[0]))
    ax.set_yticklabels(strains_s.loc[sort_index, 'name'], size=6)
    
    ax.set_ylim(data.shape[0]-0.5, -0.5)

    ax.set_xticks(np.arange(0,5000,1500)/wdw)
    ax.set_xticklabels(np.arange(0,5000,1500), size=8)
    
cbar_ax = fig.add_axes([0.75, 0.5, 0.2, 0.02])
cbar = plt.colorbar(HM, cax=cbar_ax, orientation='horizontal', label='coverage depth\nz-score')
cbar.set_ticks(np.arange(-1,4,2))
cbar.outline.set_visible(False)


plt.savefig('/home/mathieu/mhenault_landrylab/md_ty_expression/fig/fig_final/Fig2A.jpg', dpi=300)

plt.show()
plt.close()